# Why augmentation?
Simply according the [paper](https://arxiv.org/abs/1711.08324), Data augmentation is used to alleviate the over-fitting problem.

Also we should use 3d patches (sub-images) instead of whole image. This is because: "[it] is infeasible for our 3D CNN [to use entire image as input to the models] due to the GPU memory constraint. When the resolution of lung scans is kept at a fine level, even a single sample consumes more than the maximum memory of mainstream GPUs."
These patches are sized 128\*128\*128 for height, length, and width respectively.
At their paper, they have said that:

`Two kinds of patches are randomly selected. First, 70% of the inputs are selected so that they contain at least one nodule. Second, 30% of the inputs are cropped randomly from lung scans and may not contain any nodules.`

And I have covered this at `preprocess.run.save_augmented_data` file of this repository. The code is quite easy.

Anyway, I want to describe `Resize`, `Random Cropping`, and `Rotation` which are mentioned at the paper. Actually, they have not coded the `Rotation` part and just flip the image, but I thought it would be good for augmentation part if I cover it.

# Resize


In [2]:
from PIL import Image
from glob import glob
import pandas as pd
import skimage, os
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening, convex_hull_image
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border
from skimage import data
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import pydicom
import scipy.misc
import numpy as np
import SimpleITK as sitk

In [4]:
def scale(img: np.array, scale_factor: float, spacing: tuple, centers: list, radii: list):
    # original values is between 0.8 and 1.15
    assert (.75 <= scale_factor <= 1.25)
    out_centers = [tuple(np.rint(np.array(c) * scale_factor).astype(int)) for c in centers]
    out_radii = [r * scale_factor for r in radii]
    spacing = np.array(spacing) * scale_factor
    img1 = scipy.ndimage.interpolation.zoom(img, spacing, mode='nearest')
    return img1, tuple(spacing), out_centers, out_radii

To test functionality of this method, we need an argmax for 3d array. But there is no such method, so we write something like this to overcome the problem:

In [15]:
def argmax_3d(img: np.array):
    max1 = np.max(img, axis=0)
    argmax1 = np.argmax(img, axis=0)
    max2 = np.max(max1, axis=0)
    argmax2 = np.argmax(max1, axis=0)
    argmax3 = np.argmax(max2, axis=0)
    argmax_3d = (argmax1[argmax2[argmax3], argmax3], argmax2[argmax3], argmax3)
    return argmax_3d, img[argmax_3d]


I want to make an all zeros 3d image with just one 1.0 in it. This is a toy example, in the real problem, the voxel which its value is 1.0 will point to the center of a possible nodule. We made the array to track the place of our "toy nodule" after transformations!

In [46]:
img = np.zeros((100, 100, 100), dtype=float)
centers = [(60, 60, 60)] # voxel which will be 1.0
img[centers[0]] = 1.
radii = [5] # radius of our toy nodule, but here we do not need it.
spacing = (1.,1.,1.) # spacing of 3d array
rnd = np.random.random() / 2 + 0.75
img2, spacing2, centers2, radii2 = scale(img=img, scale_factor=rnd, spacing=spacing, centers=centers, radii=radii)
print(spacing2, radii)

(0.983931823135843, 0.983931823135843, 0.983931823135843) [5]


So now we wonder if resulting center is still highest value of resulting image or not. Lets see:

In [47]:
print(centers2)
print(argmax_3d(img2))

[(59, 59, 59)]
((59, 59, 59), 0.7515947186377091)


Yes! It is. but as you see, the max value is less than 1.0! why? It is because of the `scipy.ndimage.interpolation.zoom` method which we have choosen mode='nearest'. You can find documentation [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.ndimage.interpolation.zoom.html).

# Random crop

Random cropping is very important to avoid over-fitting. We must consider not cropping in a manner that nodule always stays in the center of 3d image, so this is called random cropping.

If we assume block_size as a constant, then we can assign each point inside of the original image exactly one cropping 3d cube.
Then the procedure is simple, first we should define a bounding box for the center of cropping 3d cube, then randomly choose one of the points inside that bounding box as center, and then crop 3d image!

Assume that the below picture is a ct scan which contains a nodule in it which We want to randomly crop the image in a way that it contains nodule.
![not cropped image](./figs/one_3d_cube.jpeg)




One of the cropping forms could be this one:
![cropped image](./figs/two_3d_cubes.jpeg)

Consider that if a crop falls out of the original image, it will be filled with pad_value.

In [42]:
def _get_cube_from_img_new(img, origin: tuple, block_size=128, pad_value=106.):
    assert 2 <= len(origin) <= 3
    final_image_shape = tuple([block_size] * len(origin))
    result = np.ones(final_image_shape) * pad_value
    start_at_original_images = []
    end_at_original_images = []
    start_at_result_images = []
    end_at_result_images = []
    for i, center_of_a_dim in enumerate(origin):
        start_at_original_image = int(center_of_a_dim - block_size / 2)
        end_at_original_image = start_at_original_image + block_size
        if start_at_original_image < 0:
            start_at_result_image = abs(start_at_original_image)
            start_at_original_image = 0
        else:
            start_at_result_image = 0
        if end_at_original_image > img.shape[i]:
            end_at_original_image = img.shape[i]
            end_at_result_image = start_at_result_image + (end_at_original_image - start_at_original_image)
        else:
            end_at_result_image = block_size
        start_at_original_images.append(start_at_original_image)
        end_at_original_images.append(end_at_original_image)
        start_at_result_images.append(start_at_result_image)
        end_at_result_images.append(end_at_result_image)
    # for simplicity
    sri = start_at_result_images
    eri = end_at_result_images
    soi = start_at_original_images
    eoi = end_at_original_images
    if len(origin) == 3:
        result[sri[0]:eri[0], sri[1]:eri[1], sri[2]:eri[2]] = img[soi[0]:eoi[0], soi[1]:eoi[1], soi[2]:eoi[2]]
    elif len(origin) == 2:
        result[sri[0]:eri[0], sri[1]:eri[1]] = img[soi[0]:eoi[0], soi[1]:eoi[1]]
    return result

def random_crop(img: np.array, centers: list, radii: list, main_tumor_idx: int, spacing: tuple, block_size: int,
                pad_value: float, margin: int):
    max_radius_index = np.max(np.round(radii[main_tumor_idx] / np.array(spacing)).astype(int))
    center_of_cube = list(centers[main_tumor_idx])
    shifts = []
    for i in range(len(centers[main_tumor_idx])):
        high = int(block_size / 2) - max_radius_index - margin
        if high < 0:
            high = 0
        shift = np.random.randint(low=-abs(high), high=abs(high))
        center_of_cube[i] += shift
        shifts.append(shift)
    out_img = _get_cube_from_img_new(img, origin=tuple(center_of_cube), block_size=block_size, pad_value=pad_value)
    out_centers = []
    for i in range(len(centers)):
        diff = np.array(centers[main_tumor_idx]) - np.array(centers[i])
        out_centers.append(
            tuple(np.array([int(block_size / 2)] * len(centers[i]), dtype=int) - np.array(shifts, dtype=int) - diff))
    return out_img, out_centers


Now just like previous step, we want to test if it works?

In [51]:
img3, centers3 = random_crop(img=img, centers=centers, radii=radii, main_tumor_idx=0, spacing=spacing, block_size=30, pad_value=0.0,margin=1)

In [52]:
print(argmax_3d(img3))
print(centers3[0])

((17, 19, 9), 1.0)
(17, 19, 9)


Now I want to add another center to the original image. Lets test two scenarios:
* The second center is inside the cropped image
* The second center is NOT inside the cropped image

### The second center is inside the cropped image

In [60]:
img = np.zeros((100, 100, 100), dtype=float)
centers = [(60, 60, 60), (50,50,50)]
# stands for radiuses of nodules, but here we do not need it.
radii = [5, 6]
spacing = (1.,1.,1.)
img[centers[0]] = 1.
img[centers[1]] = 1.
img3, centers3 = random_crop(img=img, centers=centers, radii=radii, main_tumor_idx=0, spacing=spacing, block_size=30, pad_value=0.0,margin=1)
print(centers3)
print([img3[centers3[0]], img3[centers3[1]]])

[(12, 14, 7), (2, 4, -3)]
[1.0, 0.0]


### The second center is NOT inside the cropped image

In [62]:
img = np.zeros((100, 100, 100), dtype=float)
centers = [(60, 60, 60), (91,91,91)]
# stands for radiuses of nodules, but here we do not need it.
radii = [5, 6]
spacing = (1.,1.,1.)
img[centers[0]] = 1.
img[centers[1]] = 1.
img3, centers3 = random_crop(img=img, centers=centers, radii=radii, main_tumor_idx=0, spacing=spacing, block_size=30, pad_value=0.0,margin=1)
print(centers3)
print([img3[centers3[0]], img3[centers3[1]]])

[(16, 16, 23), (47, 47, 54)]


IndexError: index 47 is out of bounds for axis 0 with size 30

As you can see, second tumor is not inside of the cropped 3d patch. <b>because we had mentioned that `main_tumor_idx=0`</b>, we could change it to 1, then first tumor would not be in the patch! 